# Data Preprocessing steps

V1: https://mconverter.eu/convert/docx/md/
V2: https://word2md.com/

1. First, substitute "---" with "-" in the original file - v1
2. Run the following code

In [ ]:
# path = "../data/grammar-level-1/grammar_list_word2md.md"
# clean_path = "../data/grammar-level-1/grammar_list_clean_word2md.md"

# with open(path, 'r', encoding='utf-8') as f:
#     lines = f.readlines()

# # find all indices of description lines
# desc_idxs = [i for i, L in enumerate(lines) if 'Описание' in L]
# print(len(desc_idxs))
# for idx in reversed(desc_idxs):
#     hdr_idx = idx - 2

#     # format header line as H2
#     original = lines[hdr_idx].rstrip('\n')
#     lines[hdr_idx] = f'## {original}\n'

#     # insert delimiter above it
#     lines.insert(hdr_idx, '\n---\n')

# with open(clean_path, 'w', encoding='utf-8') as f:
#     f.writelines(lines)

3. "## ###" на "##" BOTH
4. Use this regex: \{([^}]*)\} to - v1
    - Delete duplicating titles
    - Empty titles (will be fixed)
    - Delete any {mark}s
5. "** **" на " " - вроде даже не обязательно - v1
6. "ENTER >" на "ENTER \t" - v1
7. Signs with \ in the beginning: \", \~, \|, \., \*, \', \[, \], \+ BOTH
8. "--" на "-" - v1
9. "—" на "-" - v2
10. "\t-" на "-" - v2
11. "[EMSP]" на "\t" BOTH
12. Fix Long Grammar names accidentally separated. Use Obsidian table of content - easier to spot - v1
13. Use this regex \*\*([^\s\uAC00-\uD7AF])\*\* to spot weird bolding of single characters (e.g. **И**спользуется с существительными и выражает значение **«толь...) - V2
14. Add | separator between Korean and Russian grammar names BOTH


## Чекнуть \~기(가) | ~기(가) + прилагательное!

In [2]:
LEVEL = 1

def parse_entry(text):
    """
    Parse a single grammar explanation into a dictionary.
    - Separates Korean and Russian grammar names from the header.
    - Adds a predefined level.
    - Extracts "Смежные темы" into a 'related_grammar' list.
    - Stores the remaining description in the 'content' field.
    """
    lines = text.strip().splitlines()

    # --- 1. Parse Header ---
    header_line = lines[0].strip()

    # Clean the markdown: "## **이/가 именительный падеж**" -> "이/가 именительный падеж"
    clean_header = ""
    if header_line.startswith('##'):
        clean_header = header_line.replace("*", "").replace("#", "").strip()

    parts = clean_header.split(' | ', 1)
    grammar_name_kr = parts[0]
    grammar_name_rus = parts[1] if len(parts) > 1 else ""

    # --- 2. Parse Content and Related Topics ---
    content_lines = lines[1:]
    related_grammar = []

    # Trim any trailing empty lines to easily access the last content line
    while content_lines and not content_lines[-1].strip():
        content_lines.pop()

    # Check if the last non-empty line contains the related topics
    if content_lines and content_lines[-1].strip().startswith('Смежные темы:'):
        # Extract the line and remove it from the content_lines list
        related_topics_line = content_lines.pop().strip()

        # Get the string part after the colon
        topics_str = related_topics_line.split(':', 1)[1]

        # Split the topics by comma, strip whitespace from each, and filter out any empty results
        related_grammar = [topic.strip() for topic in topics_str.split(',') if topic.strip()]

    # Trim any leading empty lines from the remaining content
    while content_lines and not content_lines[0].strip():
        content_lines.pop(0)

    # Join the remaining lines to form the main content
    content = "\n".join(content_lines)

    return {
        "grammar_name_kr": grammar_name_kr,
        "grammar_name_rus": grammar_name_rus,
        "level": LEVEL,
        "related_grammars": related_grammar,
        "content": content
    }

In [3]:
def parse_input_md(input_text):
    parts = input_text.split('---')
    entries = []
    for part in parts:
        part = part.strip()
        if not part:
            continue
        entry = parse_entry(part)
        entries.append(entry)
    return entries

with open("../data/grammar-level-1/v2/grammar_list_clean_word2md.md", "r", encoding="utf-8") as infile:
    grammar_text = infile.read()

In [4]:
grammar_list = parse_input_md(grammar_text)

In [5]:
len(grammar_list)

74

In [6]:
grammar_list[0]

{'grammar_name_kr': '이/가',
 'grammar_name_rus': 'именительный падеж',
 'level': 1,
 'related_grammars': ['은/는', '께서'],
 'content': '**Описание:**\nЧастицы **이/가** обозначают **именительный падеж** и используются для указания на подлежащее в предложении. Они выполняют роль указания на **того, кто совершает действие** или **находится в определённом состоянии**. Также часто используются, когда вводится **новая информация** или подчёркивается субъект.\n\n**Форма:**\n**Существительное + 이/가**\n\n> Если существительное заканчивается на **согласную** → добавляется **이**\n\n> Если на **гласную** → добавляется **가**\n\n**Примеры:**\n학생**이** 교실에 있어요.\nСтудент находится в аудитории.\n\n고양이**가** 귀여워요.\nКошка милая.\n\n오늘**이** 제 생일입니다.\nСегодня мой день рождения.\n\n**Примечания:**\n\n1. Частицы **이/가** используются чаще всего, когда информация **новая**, в отличие от **은/는**, которые указывают на **тему разговора**.\n2. Есть несколько существительных, которые полностью изменяются в именительном па

In [18]:
import pandas as pd
import hashlib

df = pd.DataFrame(grammar_list)

def hash_text(text: str) -> str:
    return hashlib.sha256(text.encode('utf-8')).hexdigest()[:32]

df.index = df['content'].apply(hash_text)
df.index.name = "grammar_id"
df.head()

,grammar_name_kr,grammar_name_rus,level,related_grammars,content
grammar_id,,,,,
800221b78d16f6518e6a5a5b9276312f,이/가,именительный падеж,1,"[은/는, 께서]",**Описание:**\nЧастицы **이/가** обозначают **им...
71fbc9238b11089e5fc4619213435819,와/과,"«и», перечисление существительных",1,"[하고, (이)랑]",**Описание:**\nИспользуется для перечисления п...
94411eeab161532f99eb648d223c604c,와/과,"«с», совместное действие",1,"[하고, (이)랑]","**Описание:**\nУказывает на лицо или объект, с..."
47044081d8be03af2b39561ba862fa69,까지,«до»,1,"[부터, 에서 ""из""]",**Описание:**\nЧастица **까지** используется для...
c0a4df63cbf58912c1080bf8eb0707ca,께서,именительный падеж (вежл.),1,"[이/가, 께, 께서는, -(으)시-, -(으)세요]",**Описание:**\n**Это вежливая форма** именител...


## Add converting СМЕЖНЫЕ ГРАММАТИКИ to IDs

In [24]:
df.to_csv("../data/grammar-level-1/v2/grammar_list_clean_word2md.csv")
pd.read_csv("../data/grammar-level-1/v2/grammar_list_clean_word2md.csv", index_col=0).head()

,grammar_name_kr,grammar_name_rus,level,related_grammars,content
grammar_id,,,,,
800221b78d16f6518e6a5a5b9276312f,이/가,именительный падеж,1,"['은/는', '께서']",**Описание:**\nЧастицы **이/가** обозначают **им...
71fbc9238b11089e5fc4619213435819,와/과,"«и», перечисление существительных",1,"['하고', '(이)랑']",**Описание:**\nИспользуется для перечисления п...
94411eeab161532f99eb648d223c604c,와/과,"«с», совместное действие",1,"['하고', '(이)랑']","**Описание:**\nУказывает на лицо или объект, с..."
47044081d8be03af2b39561ba862fa69,까지,«до»,1,"['부터', '에서 ""из""']",**Описание:**\nЧастица **까지** используется для...
c0a4df63cbf58912c1080bf8eb0707ca,께서,именительный падеж (вежл.),1,"['이/가', '께', '께서는', '-(으)시-', '-(으)세요']",**Описание:**\n**Это вежливая форма** именител...
